<a href="https://colab.research.google.com/github/TDC2025/drive-monitor-bot/blob/main/Monitoreo_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================================
# Script de revisión diaria de cambios en Drive
# =============================================
!pip install python-telegram-bot --quiet

from google.colab import drive
import os
import json
import pandas as pd
from datetime import datetime
from pathlib import Path
import telegram
import asyncio
import time

# Marcar el inicio del proceso
tiempo_inicio = time.time()

# Montar Drive
try:
    drive.mount('/content/drive', force_remount=True)
    print("Drive remontado forzosamente para actualizar cambios")

    # Pausa para esperar sincronización completa
    print("Esperando 5 segundos para sincronización completa...")
    time.sleep(5)
except Exception as e:
    print(f"Error al montar Drive: {e}")
    exit(1)

# CONFIGURACIÓN
CARPETA_BASE = "/content/drive/MyDrive/01. Enlaces"
HISTORIAL_PATH = "/content/drive/MyDrive/Colab Notebooks/historial_drive.json"
BOT_TOKEN = "7447280502:AAGAX2boaN-lrnzCPLPoRtzFFwRlchrsj9c"
CHAT_ID = "662819001"

# FUNCIONES
def obtener_listado_archivos(carpeta_base):
    archivos_info = []
    try:
        for root, _, files in os.walk(carpeta_base):
            for file in files:
                ruta_completa = os.path.join(root, file)
                ruta_relativa = os.path.relpath(ruta_completa, carpeta_base)
                modificado = os.path.getmtime(ruta_completa)
                archivos_info.append({
                    "archivo": ruta_relativa.replace("\\", "/"),
                    "modificado": modificado
                })
        return archivos_info
    except Exception as e:
        print(f"Error al obtener listado de archivos: {e}")
        return []

# Cargar historial anterior si existe
historial_anterior = []
es_primera_ejecucion = False

try:
    if os.path.exists(HISTORIAL_PATH):
        with open(HISTORIAL_PATH, "r") as f:
            historial_anterior = json.load(f)
    else:
        es_primera_ejecucion = True
        print("No existe historial previo. Primera ejecución detectada.")
except Exception as e:
    print(f"Error al cargar historial: {e}")

# Obtener listado actual
listado_actual = obtener_listado_archivos(CARPETA_BASE)

# Convertir a DataFrame
df_anterior = pd.DataFrame(historial_anterior)
df_actual = pd.DataFrame(listado_actual)

# Comparaciones
archivos_anteriores = set(df_anterior["archivo"]) if not df_anterior.empty else set()
archivos_actuales = set(df_actual["archivo"]) if not df_actual.empty else set()

# Diagnóstico básico
print(f"Archivos en historial anterior: {len(archivos_anteriores)}")
print(f"Archivos actuales encontrados: {len(archivos_actuales)}")
print(f"Ruta de carpeta base: {CARPETA_BASE}")

# Agregados
nuevos = archivos_actuales - archivos_anteriores
# Eliminados
eliminados = archivos_anteriores - archivos_actuales

# Mostrar cambios detectados
print(f"\nNuevos archivos detectados: {len(nuevos)}")
print(f"Archivos eliminados detectados: {len(eliminados)}")

# Modificados
modificados = []
if not df_anterior.empty and not df_actual.empty:
    df_merged = df_actual.merge(df_anterior, on="archivo", suffixes=("_nuevo", "_antiguo"))

    # Filtramos para detectar solo cambios significativos (más de 1 segundo de diferencia)
    # Esto evita detectar falsos positivos por pequeñas diferencias en los timestamps
    df_modificados = df_merged[abs(df_merged["modificado_nuevo"] - df_merged["modificado_antiguo"]) > 1.0]
    modificados = set(df_modificados["archivo"])

    print(f"Archivos modificados detectados: {len(modificados)}")

    # Para debug: mostrar algunos archivos modificados si hay muchos
    if len(modificados) > 10:
        print("Mostrando solo los primeros 5 archivos modificados:")
        for idx, archivo in enumerate(sorted(modificados)):
            if idx < 5:
                print(f"  - {archivo}")
                # Mostrar la diferencia de tiempo
                row = df_modificados[df_modificados["archivo"] == archivo].iloc[0]
                diff = row["modificado_nuevo"] - row["modificado_antiguo"]
                print(f"    Diferencia de tiempo: {diff:.2f} segundos")
            else:
                break
    elif len(modificados) > 0:
        print("Archivos modificados:")
        for archivo in sorted(modificados):
            print(f"  - {archivo}")

    # Si no se detectaron cambios, forzar una verificación más detallada
    if not nuevos and not eliminados and not modificados:
        print("\nDEPURACIÓN - Verificando posibles problemas:")

        # Comprobar si realmente hay archivos en la carpeta
        if len(archivos_actuales) == 0:
            print("⚠️ ALERTA: No se encontraron archivos en la carpeta. Verifica la ruta.")

        # Comprobar si el archivo de historial está vacío o corrupto
        if len(archivos_anteriores) == 0:
            print("ℹ️ El historial anterior está vacío. Esta podría ser la primera ejecución real.")

        # Forzar actualización del historial para asegurar sincronización
        print("\n⚙️ Forzando actualización del historial para la próxima ejecución.")

        # Mostrar el contenido real de la carpeta mediante un comando directo
        try:
            import subprocess
            print("\nListado directo de la carpeta:")
            result = subprocess.run(['ls', '-la', CARPETA_BASE], capture_output=True, text=True)
            print(result.stdout)
        except Exception as e:
            print(f"No se pudo obtener listado directo: {e}")

# Preparar mensaje de Telegram solo si no es primera ejecución
if not es_primera_ejecucion:
    timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")

    # Extraer la parte más relevante de la ruta (a partir de "01. Enlaces")
    ruta_base_display = CARPETA_BASE
    if "01. Enlaces" in CARPETA_BASE:
        ruta_base_display = CARPETA_BASE[CARPETA_BASE.index("01. Enlaces"):]

    lineas = [f"📦 *Cambios detectados en:* `{ruta_base_display}`"]
    lineas.append(f"🕒 *Fecha:* `{timestamp}`")

    if nuevos:
        lineas.append("\n🆕 *Nuevos archivos:*")
        for archivo in sorted(nuevos):
            # Si el archivo no contiene '/', está en la raíz
            if '/' not in archivo:
                lineas.append(f"• `{ruta_base_display}/{archivo}`")
            else:
                # Si ya tiene estructura de carpetas, mantenerla como está
                lineas.append(f"• `{archivo}`")
    if eliminados:
        lineas.append("\n🗑️ *Eliminados:*")
        for archivo in sorted(eliminados):
            # Si el archivo no contiene '/', está en la raíz
            if '/' not in archivo:
                lineas.append(f"• `{ruta_base_display}/{archivo}`")
            else:
                # Si ya tiene estructura de carpetas, mantenerla como está
                lineas.append(f"• `{archivo}`")
    if modificados:
        lineas.append("\n✏️ *Modificados:*")
        for archivo in sorted(modificados):
            # Si el archivo no contiene '/', está en la raíz
            if '/' not in archivo:
                lineas.append(f"• `{ruta_base_display}/{archivo}`")
            else:
                # Si ya tiene estructura de carpetas, mantenerla como está
                lineas.append(f"• `{archivo}`")

    if not nuevos and not eliminados and not modificados:
        lineas.append("\n✅ Sin cambios detectados.")

    mensaje = "\n".join(lineas)

    # Función para dividir mensajes largos (Telegram tiene límite de 4096 caracteres)
    def dividir_mensaje(texto, max_length=4000):
        if len(texto) <= max_length:
            return [texto]

        partes = []
        while texto:
            if len(texto) <= max_length:
                partes.append(texto)
                break

            # Buscar un punto adecuado para dividir
            punto_corte = texto.rfind('\n', 0, max_length)
            if punto_corte == -1:
                punto_corte = max_length

            partes.append(texto[:punto_corte])
            texto = texto[punto_corte:].lstrip()

        return partes

    # ENVÍO TELEGRAM
    async def enviar_mensaje():
        try:
            bot = telegram.Bot(token=BOT_TOKEN)
            partes_mensaje = dividir_mensaje(mensaje)

            # Verificar si el bot puede comunicarse con el usuario
            try:
                # Intenta obtener información del chat para verificar que el bot tenga acceso
                chat_info = await bot.get_chat(chat_id=CHAT_ID)
                print(f"✅ Bot tiene acceso al chat: {chat_info.type}")
            except Exception as chat_error:
                print(f"⚠️ ADVERTENCIA: El bot no puede acceder al chat. Posiblemente bloqueado: {chat_error}")
                print("Por favor, asegúrate que el usuario ha iniciado una conversación con el bot.")
                print(f"El usuario debe buscar el bot y enviar un mensaje '/start'")
                return False

            # Envío del mensaje
            for parte in partes_mensaje:
                await bot.send_message(
                    chat_id=CHAT_ID,
                    text=parte,
                    parse_mode=telegram.constants.ParseMode.MARKDOWN
                )
            return True
        except Exception as e:
            print(f"Error al enviar mensaje por Telegram: {e}")
            print("\nPosibles soluciones:")
            print("1. Verifica que el token del bot sea correcto")
            print("2. Asegúrate que el usuario no ha bloqueado al bot")
            print("3. El chat_id debe ser correcto y el usuario debe haber iniciado el bot con /start")
            print("4. Si el problema persiste, crea un nuevo bot con BotFather")
            return False

    # Ejecutar función asíncrona solo si hay cambios o no es la primera ejecución
    if nuevos or eliminados or modificados:
        resultado = asyncio.run(enviar_mensaje())
        if resultado:
            print("✅ Mensaje enviado correctamente")
    else:
        print("✅ Sin cambios que reportar")
else:
    print("📝 Primera ejecución - creando historial inicial sin enviar mensaje")

# Forzar actualización del historial para asegurar que detecte cambios en la próxima ejecución
try:
    # Verificar el contenido actual del historial antes de guardar
    if os.path.exists(HISTORIAL_PATH):
        with open(HISTORIAL_PATH, "r") as f:
            historial_previo = json.load(f)
        if len(historial_previo) == len(listado_actual) and len(nuevos) == 0 and len(eliminados) == 0 and len(modificados) == 0:
            print("\n🔄 El historial parece no haber cambiado.")

            # Opción para forzar la detección de cambios en la próxima ejecución
            force_update = False  # Cambiado a False para no forzar la actualización
            if force_update:
                print("🔄 Forzando actualización del timestamp para detectar cambios en la próxima ejecución...")
                # Cambiar ligeramente los timestamps para forzar detección en la próxima ejecución
                for item in listado_actual:
                    item["modificado"] = item["modificado"] + 0.1  # Pequeño cambio en el timestamp

    # GUARDAR HISTORIAL
    with open(HISTORIAL_PATH, "w") as f:
        json.dump(listado_actual, f, indent=2)
    print("✅ Historial guardado correctamente")

    # Mostrar el estado del archivo de historial
    historial_size = os.path.getsize(HISTORIAL_PATH)
    print(f"📊 Tamaño del archivo de historial: {historial_size} bytes")

except Exception as e:
    print(f"Error al guardar historial: {e}")

# Calcular y mostrar duración total
duracion_segundos = time.time() - tiempo_inicio
minutos, segundos = divmod(duracion_segundos, 60)
print(f"⏱️ Duración del proceso: {int(minutos)} min {int(segundos)} seg")

print("✅ Revisión completa.")
